In [ ]:
import sys
from scapy.all import *
from time import *

import requests

import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Obtiene la IP con mas apariciones en una lista
def ipMasComun(ips):
    max = 0
    _ip = 0
    for k, v in ips.items():
        if v > max:
            max = v
            _ip = k
    return _ip

In [ ]:
# Obtiene el camino hacia una direccion IP
# @params:
#   max_saltos: Maximo de TTL que vamos a usar
#   num_rafagas: Cantidad de paquetes ICMP que se enviaran por TTL
#   max_time: Tiempo de espera para una respuesta

def traceroute(ip_dst, max_saltos, num_rafagas, max_time):
    responses = []
    
    for ttl in range (1, max_saltos):
        # Armamos el paquete ICMP a enviar con la IP destino y el TimeToLive
        echo_request = IP(dst=ip_dst, ttl=ttl) / ICMP()
        
        # ips: Diccionario de IPs que responden al TTL actual con su numero de respuestas
        # rtts: Diccionario de RTTs para las IPs que responden al TTL actual
        ips = {}
        rtts = {}
        for rafaga in range(num_rafagas):
            # Obtenemos la latencia entre el envio del paquete y su respuesta
            t_i = time()
            ans = sr1(echo_request, verbose=False, timeout=max_time) # envio paquete echo request
            t_f = time()
            
            if ans is not None: # Si obtenemos respuesta
                if ans.src not in ips:
                    # Si la IP es nueva se agrega
                    ips[ans.src] = 0
                    rtts[ans.src] = 0
                    
                # Agregamos respuestas para esta IP y sumamos a la latencia total
                ips[ans.src] += 1
                rtts[ans.src] += (t_f - t_i) * 1000 
                
                # Si se llega a destino termina la rafaga
                if ans.type == 0:
                    break
            else:
                # Si no se obtiene respuesta agregamos a (*) como la IP obtenida con numero de respuestas < 1
                # en caso de que si cualquier IP responde ya lo supere
                ips['*'] = 0.5
                rtts['*'] = 0
        
        # Obtenemos la IP mas comun y agregamos a la lista de responses para el TTL actual
        ip = ipMasComun(ips)
        rttPromedio = rtts[ip]/ips[ip]
        responses.append((ip, rttPromedio))

        # Vamos printeando el recorrido
        print(ttl, responses[ttl - 1])

        # Si se llega a destino se termina
        if ans is not None and ans.type == 0:
            break

    return responses

In [ ]:
# Calcula diferencia de tiempo entre saltos
def RTTEntreSaltos(response):
    rtts_dif = []
    for i in range(0, len(response) - 1):
        for j in range(i + 1, len(response)):
            rtt = response[j][1] - response[i][1]
            if rtt > 0:
                rtts_dif.append((response[i][0], response[j][0], rtt))
                break
    return rtts_dif

In [ ]:
# Pasa a un dataframe y guarda en archivo los datos y los rtts
def saveResponsesInfo(responses, testName):
    df = pd.DataFrame(responses, columns=['IPsrc', 'Latencia'])
    df.to_csv(f'results/{testName}-icmp.csv')
    
    responses = list(filter(lambda x: x[1] > 0, responses)) #filter
    rtts = RTTEntreSaltos(responses)
    df = pd.DataFrame(rtts, columns=['IPsrc', 'IPdst', 'Latencia'])
    df.to_csv(f'results/{testName}-rtts.csv')

In [ ]:
# Estudia una ruta y genera los archivos de resultados
def study_route(route_name, ip_dst, max_saltos=30, num_rafagas=8, max_time=0.8):
    route = traceroute(ip_dst, max_saltos, num_rafagas, max_time)
    saveResponsesInfo(route, route_name)

In [ ]:
filename = 'india'
website = 'www.iitd.ac.in'
ttl = 30
rafagas = 30
timeout = 0.8

In [ ]:
study_route(filename, website, ttl, rafagas, timeout)